In [ ]:
#dont touch this
optimal_arbitrary_pump = (jax_opt.minimize(opt.get_loss, arbitrary_seed_hermite_order_1,args=(N, alpha, G, H, l, y_N), method = "BFGS")).x
optimal_arbitrary_complex_pump = opt.get_complex_array(optimal_arbitrary_pump)
optimal_arbitrary_pump_cst_seed = (jax_opt.minimize(opt.get_loss, arbitrary_cst_seed,args=(N, alpha, G, H, l, y_N), method = "BFGS")).x
optimal_arbitrary_pump_cst_seed_complex = opt.get_complex_array(optimal_arbitrary_pump_cst_seed)
optimal_gaussian_theta = (jax_opt.minimize(gauss_opt.get_loss,initial_value_gauss, args = (freq_range_pump, alpha, G, H, l, y_N), method="BFGS")).x
optimal_gaussian_pump = gauss_opt.get_gaussian_pump(optimal_gaussian_theta, freq_range_pump)

In [ ]:
seed_1 = opt.get_complex_array(arbitrary_seed_hermite_order_1)
seed_2 = opt.get_complex_array(arbitrary_cst_seed)

# Test zone

In [ ]:
def penalized_obj_function(theta, freq_range, size: int, alpha, G, H, l, y_N):
    loss = opt.get_loss(theta, size, alpha, G, H, l, y_N)
    w = 1
    return loss + w*(sum(((wf-wi)/N)*freq_range*jnp.abs(theta)/jnp.linalg.norm(theta)))**2Avoiding explicit imports of NumPy is a non-goal for JAX.

In [ ]:
test_optimal_arbitrary_pump = (jax_opt.minimize(penalized_obj_function, arbitrary_seed_hermite_order_1, args=(freq_range_pump_for_real_and_complex, N, alpha, G, H, l, y_N), method="BFGS")).x

In [ ]:
dirac = np.zeros(len(freq_range_pump_for_real_and_complex))
dirac[101] = -1
dirac[101+202] = -1

In [ ]:
#penalty
plt.plot(test_optimal_arbitrary_pump)

# Plot

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(freq_range_pump[10:-10], (np.abs(optimal_arbitrary_complex_pump)/np.linalg.norm(optimal_arbitrary_complex_pump))[10:-10], label = "optimized with Hermitian polynomial multiplied by Gaussian seed")
plt.plot(freq_range_pump[10:-10], (np.abs(optimal_arbitrary_pump_cst_seed_complex)/np.linalg.norm(optimal_arbitrary_pump_cst_seed_complex))[10:-10], label = "Optimized with constant seed")
plt.plot(freq_range_pump[10:-10], (np.abs(optimal_gaussian_pump)/np.linalg.norm(optimal_gaussian_pump))[10:-10], label = "Haussian fit")
plt.plot(freq_range_pump[10:-10], (np.abs(seed_1)/np.linalg.norm(seed_1))[10:-10], label = 'Hermitian initializtion seed')
plt.plot(freq_range_pump[10:-10], (np.abs(seed_2)/np.linalg.norm(seed_2))[10:-10], label = 'Constant initializtion seed')
plt.xlabel("Normalized frequency",fontsize = 20)
plt.ylabel("Normalized absolute value of amplitude",fontsize = 20)
plt.legend(loc = 1,prop = {"size": 14})
plt.savefig("figabstract.pdf")

# Experimentation with many N values 

## Set all possible values of $\left<N\right>$

In [ ]:
y_Ns = jnp.array([0.1, 0.5, 1, 2, 3, 4, 5])

## Optimization with BFGS

In [ ]:
K_values_arbitrary = []
N_values_arbitrary = []
arbitrary_optimized_params = []
arbitrary_optimized_complex_params = []
K_values_gauss = []
N_values_gauss = []
gaussian_optimized_params = []
gaussian_optimized_complex_pump = []
for i in range(len(y_Ns)):
    optimized_arbitrary_pump = (jax_opt.minimize(opt.get_loss, arbitrary_seed_hermite_order_1, args = (N, alpha, G, H, l, y_Ns[i]), method="BFGS")).x
    arbitrary_optimized_params.append(optimized_arbitrary_pump)
    arbitrary_optimized_complex_params.append(opt.get_complex_array(optimized_arbitrary_pump))
    N_values_arbitrary.append(opt.get_observables(optimized_arbitrary_pump, N, alpha, G, H, l)[0])
    K_values_arbitrary.append(opt.get_observables(optimized_arbitrary_pump, N, alpha, G, H, l)[1])
    optimized_gaussian_params = (jax_opt.minimize(gauss_opt.get_loss, initial_value_gauss, args=(freq_range_pump, alpha, G, H, l, y_Ns[i]), method= "BFGS")).x
    gaussian_optimized_params.append(optimized_gaussian_params)
    gaussian_optimized_complex_pump.append(optimized_gaussian_params[0]*jnp.exp(-(freq_range_pump*optimized_gaussian_params[1])**2)*jnp.exp(1j*optimized_gaussian_params[2]))
    K_values_gauss.append(gauss_opt.get_observable(optimized_gaussian_params, freq_range_pump, alpha, G, H, l)[1])
    N_values_gauss.append(gauss_opt.get_observable(optimized_gaussian_params, freq_range_pump, alpha, G, H, l)[0])
    print("Finished " + str(y_Ns[i]) + " number of pairs created")

## Plotting results

In [ ]:
plt.figure(figsize = (16,8))
plt.title("Normalized optimal arbitrary pump for different N")
plt.xlabel("Frequency offset from mean frequency")
plt.ylabel("Amplitude of the pump")
for i in range(len(arbitrary_optimized_params)):
    plt.plot(w, jnp.abs(arbitrary_optimized_complex_params[i]/jnp.linalg.norm(arbitrary_optimized_complex_params[i])), label = "N = "+str(y_Ns[i]))
plt.legend()

In [ ]:
plt.figure(figsize = (16,8))
plt.title("Normalized optimal gaussian pump for different N")
plt.xlabel("Frequency offset from mean frequency")
plt.ylabel("Amplitude of the pump")
for i in range(len(arbitrary_optimized_params)):
    plt.plot(freq_range_pump, jnp.abs(gaussian_optimized_complex_pump[i]/jnp.linalg.norm(gaussian_optimized_complex_pump[i])), label = "N = "+str(y_Ns[i]))
plt.legend()

In [ ]:
plt.figure(figsize = (16,8))
plt.title("Phase of optimal arbitrary pump for different N")
plt.xlabel("Frequency offset from mean frequency")
plt.ylabel("Amplitude of the pump")
for i in range(len(arbitrary_optimized_params)):
    plt.plot(freq_range_pump, jnp.angle(gaussian_optimized_complex_pump[i]/jnp.linalg.norm(gaussian_optimized_complex_pump[i])), label = "N = "+str(y_Ns[i]))
plt.legend()

In [ ]:
plt.plot(np.array(gaussian_optimized_params).T[1])

In [ ]:
K_values_gauss, K_values_arbitrary